# Q1

In [107]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Part(a)
#### Code up the perceptron algorithm described on slide 7 of Lecture 15 using the same notation as in the slides.  [10 points]

- define the function that will be used in next sextions

In [108]:
# to reproduce results
np.random.seed(1)
'''
possible to give x,y in one 2D array then some indexing changes needs to be done
input: hyper-parameters and data wit labels
output: perceptron learned weight vector
'''
def percep(feature_data,labels,margin=0,learn_rate=1,n_iter=100):
    # convert class labels into +1/-1
    # 0 --> -1
    # 1 --> +1

    class_label=[]
    for c in labels:
        if c==0:
            class_label.append(-1)
        else:
            class_label.append(1)
    # print(class_label)
    # get feature length and total no of samples
    N,d=feature_data.shape

    # intialize w
    # with w=zeros it get stuck in the for loop
    w=np.ones(d)
    t=0
    # for loop
    for i in range(n_iter):
        '''
        break while loop when mistake condition satisfies in if statement
        '''
        while(True):
            # If `high` is None (the default), then results are from [0, `low`).
            rand=np.random.randint(N)
            curr_x=feature_data[rand]
            curr_y=class_label[rand]

            # mistake condition
            if (curr_y*np.dot(w.T,curr_x)<margin): # margin =0(default)
                w=w+curr_x*curr_y*learn_rate # learn_rate=1(default)
                # print(f'mistake -- {w}')
                break

    return w

## Part(b)

#### Write functions to make predictions using the algorithm for the banknotes dataset. Preprocess the dataset to handle missing and anomalous data

### read data

In [109]:
# easy way to get txt into csv using pandas read_table
cols=['variance','skewness','curtosis','entropy-of-image','class']
data=pd.read_table('datasets/data_banknote_authentication.txt',sep=',',names=cols)

#### pre-process check

- points with variance less than zero ignored can be ignored

In [110]:
data.isnull().sum()
# no missing data

variance            0
skewness            0
curtosis            0
entropy-of-image    0
class               0
dtype: int64

- no missing data in banknote dataset

### predict and f1-score func

In [111]:
'''
takes a new-test point and
returns its class using
# 0 --> -1
# 1 --> +1
'''
def predict(x,w):
    pred=np.dot(w.T,x)
    if pred > 0:
        return 1
    else:
        return 0

In [112]:
'''
inputs: true and predicted labels; both inputs are lists
output: return f1-score
All formulas are used from slides
'''
def f1_score(y_true,y_pred):
    # initiate variables
    tp=0
    fp=0
    fn=0

    for i in range(len(y_pred)):
        # true positives cond
        if y_pred[i]==y_true[i] and y_true[i]==1:
            tp+=1
        # false negatives cond
        elif y_true[i]==1 and y_pred[i]==0:
            fn+=1
        # false positives cond
        elif y_true[i]==0 and y_pred[i]==1:
            fp+=1

    # calculate stats
    p=tp/(tp+fp)
    r=tp/(tp+fn)

    # calc f1-score from stats(p & r)
    f1=(2*p*r)/(p+r)

    return round(f1,3) # round up-to only 3 decimal points

In [113]:
y_pred=[predict(p,w) for p in f]

In [114]:
print(f'F1 Score is >> {f1_score(y,y_pred)}')

F1 Score is >> 0.906


## Part(c)

#### Train the algorithm on the dataset using cross-validation and report cross-validated test set error 

In [115]:
'''
input: total observed points, no of folds, random state
output: shuffled list of with fold no in a tuple(so that points can be identified)
'''
def k_fold_splitter(obs_no,k=10):
    
    # suffle points
    np.random.seed(1)
    points=list(range(obs_no))
    np.random.shuffle(points)

    factor=obs_no//k

    # init split folds store list
    k_fold_list=list()

    # k-1 folds are of equal size
    # fold count starts from 0 and goes upto (k-1)
    for i in range(k-1):
        for j in range(factor):
            k_fold_list.append((points[j],i))
    
    counted_no=len(k_fold_list)

    # last fold holds everything that is remaining
    for i in range(obs_no-len(k_fold_list)):
        k_fold_list.append((points[i+counted_no],k-1))

    return k_fold_list

#### cross-validation(k-fold) steps followed:

- first split data into k-folds using the function defined above
- repeat these two steps until each of the k-folds has served as the test set
    - pick one as test and others as train folds and 
    - report testing f1 score for that run
- the average of your k recorded scores is called the cross-validation score and will serve as your performance metric for the model

In [116]:
'''
simple function to cross validate
steps explined in above cell
'''
def cross_valid(folds,margin=0,learn_rate=1,n_iter=100):

    np.random.seed(1)
    global data

    split_tuple_list=k_fold_splitter(data.shape[0],k=folds)

    f1_list=list()
    weight_list=list()

    for i in range(folds):

        fold_point_list=[]
        test_point_list=[]

        for point in split_tuple_list:
            if point[1]!=i:
                fold_point_list.append(point[0])
            else:
                test_point_list.append(point[0])

        # training data
        train_fold_data=data.iloc[fold_point_list,:]
        train_y=list(train_fold_data['class'])
        train_x=np.array(train_fold_data.drop('class',axis=1))

        # testing data
        test_fold_data=data.iloc[test_point_list,:]
        test_y=list(test_fold_data['class'])
        test_x=np.array(test_fold_data.drop('class',axis=1))

        # train
        w=percep(train_x,train_y,margin,learn_rate,n_iter) # with default values
        weight_list.append(w)
        
        # predict on test fold points
        y_pred=[predict(new_point,w) for new_point in test_x]

        # calc f1-score using the function
        f1=f1_score(test_y,y_pred)

        # store f1-score of the test ith fold
        f1_list.append(f1)

        # print(f'Testing F1 score for test fold {str(i+1)} is {f1}')

    return {'f1-scores':f1_list,'weights':weight_list}

In [117]:
folds=10
# using default values
cross_data=cross_valid(folds)
print(f'avg F1 score(cross-validation score) is {round(sum(cross_data["f1-scores"])/folds,3)}')

avg F1 score(cross-validation score) is 0.879


## Part(d)

#### Ensure you use a held out validation set and report F1 score on the held out set for your best model

#### best model strategy implemented

- there are mainly 3 hyperparameters in perceptron algo
- by changing them one by one we decide which one is best others are held constant [we also could do grid search but takes much more time]
- then on best model parameters we give avg cross-validated test set F1 score

In [118]:
# model avg f1-score for diff # of iters
for n in [100,1000,10000]:
    cross_data=cross_valid(folds,n_iter=n)
    print(f'# of iterations={n} || avg F1 score is {round(sum(cross_data["f1-scores"])/folds,3)}')

# of iterations=100 || avg F1 score is 0.879
# of iterations=1000 || avg F1 score is 0.89
# of iterations=10000 || avg F1 score is 0.912


- there is slight improvment but not quite significant
    - 10000 gives best out of these

In [119]:
# model avg f1-score for diff learning rates
for m in list(np.linspace(0,1,10)):
    cross_data=cross_valid(folds,learn_rate=m)
    print(f'learn rate={round(m,2)} || avg F1 score is {round(sum(cross_data["f1-scores"])/folds,3)}')

learn rate=0.0 || avg F1 score is 0.19
learn rate=0.11 || avg F1 score is 0.853
learn rate=0.22 || avg F1 score is 0.867
learn rate=0.33 || avg F1 score is 0.891
learn rate=0.44 || avg F1 score is 0.884
learn rate=0.56 || avg F1 score is 0.861
learn rate=0.67 || avg F1 score is 0.889
learn rate=0.78 || avg F1 score is 0.823
learn rate=0.89 || avg F1 score is 0.91
learn rate=1.0 || avg F1 score is 0.879


- there is slight up-downs but no drastic improvements-drops
    - 0.89 gives best out of these

In [120]:
# model avg f1-score for diff margins
for m in list(np.linspace(0,1,10)):
    cross_data=cross_valid(folds,margin=m)
    print(f'margin={round(m,2)} || avg F1 score(cross-validation score) is {round(sum(cross_data["f1-scores"])/folds,3)}')

margin=0.0 || avg F1 score(cross-validation score) is 0.879
margin=0.11 || avg F1 score(cross-validation score) is 0.879
margin=0.22 || avg F1 score(cross-validation score) is 0.856
margin=0.33 || avg F1 score(cross-validation score) is 0.886
margin=0.44 || avg F1 score(cross-validation score) is 0.874
margin=0.56 || avg F1 score(cross-validation score) is 0.899
margin=0.67 || avg F1 score(cross-validation score) is 0.891
margin=0.78 || avg F1 score(cross-validation score) is 0.899
margin=0.89 || avg F1 score(cross-validation score) is 0.898
margin=1.0 || avg F1 score(cross-validation score) is 0.89


- no significant changes
    - we choose 0 that was the default

In [121]:
best_margin=0
best_rate=0.89
best_n_iters=10000

cross_data=cross_valid(folds,margin=best_margin,learn_rate=best_rate,n_iter=best_n_iters)
print(f'for best model avg F1 score on held-out data is {round(sum(cross_data["f1-scores"])/folds,3)}')

for best model avg F1 score on held-out data is 0.906


# Q2

#### Let’s consider a simple demonstration of MCMC sampling in a setting where conjugacy is actually possible – normal likelihoods with a known population variance, for which the prior is another normal distribution

## Part(a)

#### Write a function to calculate the Bayesian posterior probability given 50 new data samples drawn from a normal distribution with mean 10 and SD 5, assuming a normal prior with mean 25 and s.d. 5. Plot the pdfs of the prior, the likelihood and the posterior distributions. Explain how you derive the likelihood from the data.

In [122]:
'''
this takes likelihood and prior parameters as input
and returns posterior mean and sd using the formulas 
their derivation is explained in markdown comments
'''
def posterior(n_samples=50,lik_mean=10,lik_sd=5,prior_mean=25,prior_sd=5):
    # to reproduce random state is fixed
    np.random.seed(1)
    samples=np.random.normal(lik_mean,lik_sd,n_samples)
   
    # formulas derivations for these calc explained in markdowns

    post_sd=np.sqrt((lik_sd**2*prior_sd**2)/(lik_sd**2+prior_sd**2))

    post_mean=((lik_sd**2)*(prior_mean)+(n_samples*prior_sd**2)*(np.mean(samples)))/((n_samples*prior_sd**2)+lik_sd**2)

    return {'post_mean':round(post_mean,2),'post_sd':round(post_sd,3)}

In [123]:
'''
takes mean and sd of an normal dist
and returns 
    - a equi space points around mean with 3*sd width and 
    - its pdf value for given normal dist in a dictionary 
'''
def norm_data(mean,sd):
    x=np.linspace(-3*sd+mean,mean+3*sd,100)
    norm_pdf = (1/(np.sqrt(2*np.pi) * sd )) * np.exp(-0.5*((x-mean)/sd)**2)
    return {'x':list(x),'pdf':list(norm_pdf)}

#### To plot lik-prior-posterior data 

In [124]:
# init given info
n_samples=200
lik_mean=10
lik_sd=5
prior_mean=25
prior_sd=5

# call posterior function to get params values with default values
post=posterior()

# get x-y data for plots
lik_data=norm_data(lik_mean,lik_sd)
prior_data=norm_data(prior_mean,prior_sd)
post_data=norm_data(post['post_mean'],post['post_sd'])

# plot pdf compare graph
plt.plot(lik_data['x'],lik_data['pdf'],label='lik',color='black')
plt.plot(prior_data['x'],prior_data['pdf'],label='prior',color='red')
plt.plot(post_data['x'],post_data['pdf'],label='posterior',color='green')
plt.legend()
plt.title("Density plot for lik, prior, and posterior")
plt.ylabel('pdf-values')
plt.xlabel('x')
plt.savefig('q2-a.png')
plt.show()

### observations:
- for these specific values
    - posterior mean has shifted in the direction of prior only by a very small amount(0.17) and 
    - sd has dropped to 3.536 from 5 that was for likelihood and prior.

### `Explain Likelihood Derivation from Data`

We basically have to derive the likehood from the samples we generated in `posterior` function. We will consider a general situation with mean $\mu$ and sd $\sigma$. Our data is just a specific case of that.

Now, consider N i.i.d. scalar obs $X = \{x_1 , x_2 , \ldots , x_N \}$ drawn from a normal distribution with mean $\mu$ and sd $\sigma$. So, $\forall x \in \{1,\ldots,N\}$ we have :
$$
p(x_n|\mu,\sigma^2)=\mathcal{N}(x_n|\mu,\sigma^2)
$$

Or we can write,
$$
p(x_n|\mu,\sigma^2)  = \frac{1}{\sqrt{2\pi \sigma^2}}exp\left[-\frac{(x_n-\mu)^2}{2\sigma^2}\right]
$$

Using above we can give joint likelihood simply as their product(as they are iid),
$$
p(X|\mu,\sigma^2)  = \prod_{n=1}^N p(x_n|\mu,\sigma^2)
$$

Full expanded version can be given as,
$$
p(X|\mu,\sigma^2)  = \frac{1}{(2\pi)^{n/2} \sigma^n}exp\left[-\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2\right]
$$


### `posterior derivation formulas explained`

To get posterior for `mean`($\mu$) from lik and prior we can use bayes formula,
$$
p(\mu|X)=\frac{p(X|\mu)p(\mu)}{p(X)}=\frac{p(X|\mu)p(\mu)}{\int p(X|\mu)p(\mu)}
$$

Up to proportinality we write(as integral is just a constant ),
$$
p(\mu|X) \propto p(X|\mu)p(\mu)
$$

**`Lik:`**

Given in above section. We will ignore constants(w.r.t. to $\mu$) though.

**`Prior:`**

For general case when prior for $\mu$ is a Guassian with mean $\mu_0$ and sd $\sigma_0$, we can write prior distribution,
$$
p(\mu|\mu_0,\sigma_0^2)  = \frac{1}{\sqrt{2\pi \sigma_0^2}}exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]
$$

**`Posterior:`**

Use Baues rule upto constant,
$$
p(\mu|X) \propto \left(
    \frac{1}{(2\pi)^{n/2} \sigma^n}exp\left[-\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2\right]\right) \left(\frac{1}{\sqrt{2\pi \sigma_0^2}}exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]\right)
$$

WE also can ignore other constants like $\sigma$(as it si known) and $\sigma_0$,
$$
p(\mu|X) \propto \left(
    exp\left[-\frac{1}{2\sigma^2} \sum_{n=1}^N (x_n-\mu)^2\right]\right) \left(exp\left[-\frac{(\mu-\mu_0)^2}{2\sigma_0^2}\right]\right)
$$

After using square trick and comaring with
$$
p(\mu|X)  \propto exp\left[-\frac{(\mu-\mu_N)^2}{2\sigma_N^2}\right]
$$

we get Gaussian posterior’s precision as the sum of the prior’s precision and sum of the noise
precisions of all the observations,
$$
\frac{1}{\sigma_N^2}=\frac{1}{\sigma_0^z} + \frac{N}{\sigma^2}
$$

and Gaussian posterior’s mean is a convex combination of prior’s mean and the MLE solution,
$$
\mu_N=\frac{\mu_0 \sigma^2+\bar{x}N\sigma_0^2}{N\sigma_0^2+\sigma^2}
$$

Here: $\bar{x}=\frac{\sum_{n=1}^N x_n}{N}$

NOTE: these formulas have been used in `posterior()` function to get posterior mean and SD parameter values


## Part(b)

#### Implement the Metropolis algorithm from the lecture slides to estimate the posterior distribution given the same prior and data and show that it converges to the analytic posterior by plotting a histogram of samples from the distribution alongside the analytic posterior distribution.  Assume whatever SD (width) you want for the proposal distribution.

### `Metropolish Hastings Algorithm`

Now suppose that we did not know about square trick then it won't have been easy to do this integral and compute above posterior in closed and nice form. And that happens all the time in real worl dproblem. So, we are going to experiment with this problem where we suppose we don't know how to solve above problem and we will use Monte Carlo Markov Chains to get enough samples from the posterior distribution formula up to prop constant and we call this our `Target` distribution from which we need samples but we can not as it is not a standard known dist. That is why we use a proposal dist(TBD) that conditions on the immediately previous $\mu$ value (say a Gaussian).

Here we have to decide what to use as our proposal(jump) distribution. Since we know our posterior(target dist) has support all over the real line, we can consider Univariate Norrmal Distribution for this case as it does cover real line and also simple sample from. 

So our proposal distribution is a Normal Distribution. 

**`Algo Steps:`** A general MH algo steps are,

Let $x_0 =init$ and for a $k^{th}$ step when $x_k=x$ and to obtain $x_{k+1}$ we do these steps:
1. Sample a new possible proposal $y$ from the jump distribution that conditions on the immediately previous (that is $x$ here) value.
2. Calculate the ratio of the proposed posterior distribution to the current one. The ratios is given by
$$
r=\frac{f(y)g(x|y)}{f(x)g(y|x)}
$$

But since we are using guassian dist as our proposal we have $g(x|y)=g(y|x)$. meaning it is symmatric. We have same probability of going from x to y as y to x. So, remember here f is our posterior distribution upto prop constant,
$$
r=\frac{f(y)}{f(x)}
$$

3. Sample a random number a between 0 and 1 using `Uniform dist` --> $U(0,1)$. Say the sample is `a`
4. If r > a, accept the proposed $y$, otherwise stick with the earlier $y$
5. This gives one sample from the target distribution. We can repeat above steps to get more samples.

In [125]:
'''
function to compute target dist's probability value in each run
takes a point from support space(real line here)
returns log of the posterior dist at that point
we have used log for computational stability(it does not affect inequality condition for choosing samples from MCMC as it is monotionic)
'''
def log_post(x,mean,sd):
    # log og posterior up to prop constant
    log_p=-((x-mean)/sd)**2
    
    return log_p

In [126]:
'''
takes hyperparameter values and
returns mcmc samples
other things explained in func comments
'''
no_of_samples=10000
def mh(jump_sd=5,init=10,no_of_samples=1000):
    global post
    # to reproduce results
    np.random.seed(1)

    # list to store samples
    mcmc_samples=list()

    x_curr=init
    '''
    x_curr=10 # inital value --> hyper-para
    no_of_samples=10000 # hyper-para
    jump_sd=5 # hyper-para
    '''

    acc_count=0
    for i in range(no_of_samples):
        # propose a new value from jump dist
        y_prop=np.random.normal(x_curr,jump_sd)

        # calc ratio( in log ) --> posterior parameters
        r=(log_post(y_prop,post['post_mean'],post['post_sd']))-(log_post(x_curr,post['post_mean'],post['post_sd']))

        # propose random sample from uniform
        u=np.random.uniform(0,1)

        # accept-reject condition for MH
        if r > np.log(u):
            mcmc_samples.append(y_prop)
            # update value of current sample
            x_curr=y_prop
            # update acceptance count
            acc_count+=1
        else:
            mcmc_samples.append(x_curr)
    
    print(f'acceptance rate is {acc_count*100/no_of_samples} with jump SD={jump_sd}')
    return mcmc_samples


# actual posterior params that we got in first que is used
actual_samples=np.random.normal(post['post_mean'],post['post_sd'],no_of_samples)
# func return
mcmc_samples=mh(no_of_samples=no_of_samples)
# plot hists to compare
plt.hist(mcmc_samples,label='estimate(MCMC)',bins=100,alpha=0.3,density=True)
plt.hist(actual_samples,label='analytic',bins=100,alpha=0.3,density=True)
plt.legend()
plt.title("hist plot for mcmc and posterior(actual) samples")
plt.xlabel('x')
plt.savefig('q2-b.png')
plt.show()

acceptance rate is 50.3 with jump SD=5


## Part(c)

#### How does the speed of convergence of the sampling depend on the proposal width? Is there an optimal proposal width that would work best? Demonstrate the consequences of using sub-optimal proposal width and terminating sampling too soon.

- we can measure spped of convergance by acceptance rate; when for fixed no of iterations and different jump SDs acceptance rate is low we know that algo will take more time to converge and vice-versa.

In [127]:
# with 10000 iterations being fixed
for sd in [1,2,3,5,7,10]:
    mcmc_samples=mh(jump_sd=sd,no_of_samples=no_of_samples)

acceptance rate is 86.74 with jump SD=1
acceptance rate is 75.76 with jump SD=2
acceptance rate is 65.44 with jump SD=3
acceptance rate is 50.3 with jump SD=5
acceptance rate is 39.48 with jump SD=7
acceptance rate is 29.4 with jump SD=10


#### obervatios on convergance speed

- If SD is high, then we would have more rejections and thus we would have covered less unique points in the support space, that means we would need more iterations to cover the support space; that implies that we would need more time.
- When SD is low reverse thing happens, we are moving quite quckely as acceptance rate is high and we cover space quickely and in less no of iterations.

`so speed of convergance depends inversely on the proposal width`

### Observations on optimal width for proposal

From the above example it is clear that the quality of the MCMC sampler is determined by the choice of the variance in the proposal distribution(SD).
- If SD is too large, then we will propose values potentially too far away, which in principle would be good, but this means a lot of the values will be rejected. That means we will stay where we are quite often, increasing the correlation(samples would be highly corr with their previous values as most of the time we would not even move) in the samples
- If SD is too small, then we will make tiny moves implying we may accept a lot of them, but the samples obtained will be quite dependent.


#### bad mcmc

- very less no of iterations are given to the model
- also either very high or very low SD is given to the model

- these situations can lead to very bad samples when given inital value far off from mean of actual posterior

- with high SD

In [128]:
no_of_samples=1000
# actual posterior params that we got in first que is used
actual_samples=np.random.normal(post['post_mean'],post['post_sd'],no_of_samples)
# func return
mcmc_samples=mh(jump_sd=200,init=-10,no_of_samples=no_of_samples)
# plot hists to compare
plt.hist(mcmc_samples,label='estimate(MCMC)',bins=100,alpha=0.3,density=True)
plt.hist(actual_samples,label='analytic',bins=100,alpha=0.3,density=True)
plt.legend()
plt.title("hist plot for mcmc and posterior(actual) samples")
plt.xlabel('x')
plt.savefig('q2-c1.png')
plt.show()

acceptance rate is 2.1 with jump SD=200


- with very low SD(although acceptance rate is high but of no use as we are not exploring in right place when initalization is very bad)

In [129]:
no_of_samples=1000
# actual posterior params that we got in first que is used
actual_samples=np.random.normal(post['post_mean'],post['post_sd'],no_of_samples)
# func return
mcmc_samples=mh(jump_sd=0.01,init=0,no_of_samples=no_of_samples)
# plot hists to compare
plt.hist(mcmc_samples,label='estimate(MCMC)',bins=100,alpha=0.3,density=True)
plt.hist(actual_samples,label='analytic',bins=100,alpha=0.3,density=True)
plt.legend()
plt.title("hist plot for mcmc and posterior(actual) samples")
plt.xlabel('x')
plt.savefig('q2-c2.png')
plt.show()

acceptance rate is 99.3 with jump SD=0.01


- with nice values

In [130]:
no_of_samples=1000
# actual posterior params that we got in first que is used
actual_samples=np.random.normal(post['post_mean'],post['post_sd'],no_of_samples)
# func return
mcmc_samples=mh(jump_sd=5,init=10,no_of_samples=no_of_samples)
# plot hists to compare
plt.hist(mcmc_samples,label='estimate(MCMC)',bins=100,alpha=0.3,density=True)
plt.hist(actual_samples,label='analytic',bins=100,alpha=0.3,density=True)
plt.legend()
plt.title("hist plot for mcmc and posterior(actual) samples")
plt.xlabel('x')
plt.savefig('q2-c3.png')
plt.show()

acceptance rate is 50.9 with jump SD=5


- although acceptance rate is not that good but also not that bad
- even with 1000 iterations gives nice enough picture about actual posterior
- other cases also can be considered when parameter values are not that good